In [1]:
%pip install bs4 selenium webdriver_manager

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=9e08a9cf5e894db54c2f5b591a22f3b942d59920ce8e9071d363bff28ebd3c81
  Stored in directory: c:\users\lucet\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [150]:
import pandas as pd

df_bus = pd.read_csv('bus.csv', encoding='euc-kr')
df_subway = pd.read_csv('subway.csv', encoding='euc-kr')
df_bike = pd.read_csv('bike.csv', encoding='cp949')

In [65]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

time_delta = 0.2
class KakaoRouteFinder:
    def __init__(self):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def find_routes(self, origin, dest):
        print(origin, dest)

        base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target=bike&sName={origin}&eName={dest}"
        self.driver.get(base_url)
        time.sleep(time_delta)

        self.driver.find_element(by=By.CSS_SELECTOR, value="body").click()
        time.sleep(time_delta)
        
        try:
            dimmed_layer_selector = "#dimmedLayer"
            self.driver.find_element(
                by=By.CSS_SELECTOR,
            value=dimmed_layer_selector
            ).click()
        except ElementNotInteractableException:
            pass

        def fetch_bike():
            print(" > bike")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(1)").click()        
            time.sleep(time_delta)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(2)").click()        
            time.sleep(time_delta)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(3)").click()        
            time.sleep(time_delta)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView")
            return element.get_attribute('innerHTML')
        
        def fetch_walk():
            print(" > walk")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#walktab").click()
            time.sleep(time_delta)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.WalkRouteResultView")
            return element.get_attribute('innerHTML')
            
        def fetch_transit():
            print(" > transit")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#transittab").click()
            time.sleep(2)
            
            for route in self.driver.find_elements(by=By.CSS_SELECTOR, value=f"li.TransitRouteItem"):
                route.click()
                for more_route in route.find_elements(by=By.CSS_SELECTOR, value=f"span.moreBtn"):
                    more_route.click()
                    time.sleep(time_delta)
                time.sleep(time_delta)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"ul.TransitTotalPanel")
            return element.get_attribute('innerHTML')

        try:
            return {
                # 'bike': fetch_bike(),
                # 'walk': fetch_walk(),
                'transit': fetch_transit()
            }
        except Exception as e:
            print(e)

    def __del__(self):
        self.driver.close()

routes = KakaoRouteFinder().find_routes('대한건축학회', '서울대학교 자하연')
# routes = KakaoRouteFinder().find_routes('서울시청', '서울고속터미널')



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


대한건축학회 서울대학교 자하연
 > transit


### Bike

In [45]:
soup = BeautifulSoup(routes['bike'], 'html.parser')
for route in (soup.find_all('li', {"class": "BikeRouteItem"})):
    res = {
        "mode": route.find('span', {"class": "mode"}).text,
        "time": route.find('span', {"class": "time"}).text.strip(),
        "distance": route.find('span', {"class": "distance"}).text,
        "altitude": route.find('span', {"class": "altitude"}).text,
        "calories": route.find('span', {"class": "calories"}).text,
    }
    print(res)

{'mode': '자전거 도로우선', 'time': '38분', 'distance': '6.2km', 'altitude': '고도 최저 17m, 최고 135m', 'calories': '229kcal'}
{'mode': '최단거리', 'time': '38분', 'distance': '6.1km', 'altitude': '고도 최저 17m, 최고 135m', 'calories': '226kcal'}
{'mode': '편안한길', 'time': '39분', 'distance': '6.4km', 'altitude': '고도 최저 17m, 최고 128m', 'calories': '233kcal'}


### Transit

In [43]:
soup = BeautifulSoup(routes['transit'], 'html.parser')
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    res = {
        "time": route.find('span', {"class": "time"}).text.strip(),
        "info": route.find('span', {"class": "walkTime"}).get('title'),
    }
    print(res)

{'time': '35분', 'info': '도보17분 | 환승1회 | 요금 1,250원 | 7.8km'}
{'time': '40분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 7.2km'}
{'time': '39분', 'info': '도보19분 | 환승1회 | 요금 1,250원 | 7.7km'}
{'time': '39분', 'info': '도보20분 | 환승1회 | 요금 1,250원 | 8.3km'}
{'time': '43분', 'info': '도보11분 | 환승1회 | 요금 1,200원 | 7.7km'}
{'time': '42분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 7.5km'}
{'time': '44분', 'info': '도보16분 | 환승1회 | 요금 1,200원 | 7.4km'}
{'time': '40분', 'info': '도보24분 | 환승1회 | 요금 1,250원 | 7.5km'}
{'time': '47분', 'info': '도보14분 | 환승1회 | 요금 1,200원 | 8.2km'}
{'time': '42분', 'info': '도보26분 | 환승1회 | 요금 1,250원 | 7.6km'}


### Walk

In [44]:
soup = BeautifulSoup(routes['walk'], 'html.parser')
for route in (soup.find_all('li', {"class": "WalkRouteItem"})):
    res = {
        "mode": route.find('span', {"class": "mode"}).text,
        "time": route.find('span', {"class": "time"}).text.strip(),
        "info": route.find('div', {"class": "info"}).text.strip(),
    }
    print(res)

### Coordinate Transform

In [39]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [24]:
import requests
import dotenv
from os import environ

dotenv.load_dotenv('.env', override=True)

def transform(coord, input_coord='WCONGNAMUL', output_coord='WGS84'):
    URL = f'https://dapi.kakao.com/v2/local/geo/transcoord.json?x={coord[0]}&y={coord[1]}&input_coord={input_coord}&output_coord={output_coord}'
    headers = {'Authorization': f'KakaoAK {environ.get("KAKAO_REST_API_KEY")}'}
    new_coord = requests.get(URL, headers=headers).json()['documents'][0]
    return (new_coord['x'], new_coord['y'])

transform((495285,1129803), 'WCONGNAMUL', 'WGS84')

(126.97865225830876, 37.56682600466712)

In [26]:
(org, des) = ('org', 'des')
(org_x, org_y) = transform((126.97865225830876, 37.56682600466712), 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = transform((127.00515186540567, 37.49275468135762), 'WGS84', 'WCONGNAMUL')
print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=495285,1129803,501139,1109250&rt1=org&rt2=des


https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=495285,1129803,492630,1108280&rt1=org&rt2=des

In [91]:
soup = BeautifulSoup(routes['transit'], 'html.parser')

S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

In [113]:
%pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [162]:
from haversine import haversine

def get_distance(org, des):
    return haversine(org, des, unit='m')
    
get_distance((37.550623,	126.914980), (37.550623,	126.914984))

0.35262858573738154

In [184]:
def get_nearest_bike(lat, lng):
    delta = 5e-3
    df = df_bike[(df_bike["위도(Y)"] > lat-delta) & (df_bike["위도(Y)"] < lat+delta) \
               & (df_bike["경도(X)"] > lng-delta) & (df_bike["경도(X)"] < lng+delta) ]

    min_bike, min_dist = "N/A", float("inf")
    for i, bike in df.iterrows():
        dist = get_distance((lat, lng), bike[["위도(Y)", "경도(X)"]])
        if min_dist > dist:
            min_dist = dist
            min_bike = bike
    return min_dist, min_bike

get_nearest_bike(37.550007,	126.914825)

(0.03811098653084432,
 대여소\n번호                    105
 보관소(대여소)명           합정역 5번출구 앞
 자치구                        마포구
 상세주소          서울특별시 마포구 양화로 48
 위도(Y)                37.550007
 경도(X)               126.914825
 설치\n시기              2015-09-06
 설치형태                       5.0
 Unnamed: 8                 NaN
 운영\n방식                     LCD
 Name: 1369, dtype: object)

In [189]:
for s in S:
    # print(s)
    bus = df_bus[df_bus['정류소명'] == s]
    subway = df_subway[df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
            print(s, lat, lng, get_nearest_bike(float(lat), float(lng))[1]['보관소(대여소)명'])
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
            print(s, lat, lng, get_nearest_bike(float(lat), float(lng))[1]['보관소(대여소)명'])
        else:
            print(s, 'N/A')
    except:
        print('N/A')
    # print(df_bus[df_bus['정류소명']==s])

사당1동관악시장앞 37.4767901 126.977952 광일빌딩
N/A
낙성대입구 37.47790968 126.9618757 낙성대역 5번출구
N/A
관악경찰서.관악소방서 37.475253 126.952572 관악구청교차로
서울대후문.연구공원 37.46505396 126.9576576 낙성대 교수아파트
서울과학전시관 37.46872249 126.9579992 낙성대 과학전시관
예술인마을 37.47604923 126.9728451 광일빌딩
가족생활동 37.4669274 126.9577616 낙성대 교수아파트
국제대학원 37.46417722 126.9553135 낙성대 교수아파트
봉천사거리.봉천중앙시장 37.48288738 126.9542258 중앙동 동진빌딩
호암교수회관 37.46841173 126.958334 낙성대 과학전시관
낙성대공원.영어마을 37.47217216 126.9586485 영어마을 관악캠프
서울교통공사 37.47671115 126.9861652 연세사랑병원신관앞
N/A
서울여상.서울문영여중고앞 37.47316048 126.9533001 낙성대 교수아파트
서울대학교 37.46674146 126.9479523 서울대학교 정문
서울대입구역 37.481247 126.952739 서울대입구역 1번출구
법대.사회대입구 37.46310498 126.9488366 서울대학교 정문
낙성대현대아파트 37.47487295 126.9591501 인헌초교
방배역 37.481426 126.997596 방배역 2번 출구
사당역 37.47653 126.981685 사당역 5번출구
행운동주민센터 37.47984444 126.956869 사랑의병원
서울미술고.인헌중고 37.47646634 126.9655544 JK장평타워
봉천사거리.위버폴리스 37.48279341 126.9535723 중앙동 동진빌딩
인헌아파트 37.46802075 126.9612872 낙성대 과학전시관
경영대 37.466135 126.952053 서울대학교 정문
관악구청 37.47798198 126.952

In [32]:
df_bus

,ARS-ID,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987750,37.569765
1,1002,창경궁.서울대학교병원,126.996566,37.579183
2,1003,명륜3가.성대입구,126.998340,37.582671
3,1004,종로2가.삼일교,126.987613,37.568579
4,1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...,...
11310,25995,우성아파트,127.139338,37.550386
11311,25996,우성아파트,127.140046,37.550643
11312,25997,조일약국,127.123596,37.533630
11313,25998,성내시장,127.125497,37.536155


In [30]:
df_subway[['code', 'name', 'lat(y)', 'lng']]

,code,name,lat(y),lng
0,1908.0,녹양,37.759380,127.042292(x)
1,1002.0,남영,37.541021,126.9713
2,1003.0,용산,37.529849,126.964561
3,1004.0,노량진,37.514219,126.942454
4,1005.0,대방,37.513342,126.926382
...,...,...,...,...
725,3135.0,지식정보단지,37.378384,126.645168
726,3136.0,인천대입구,37.386007,126.639484
727,3137.0,센트럴파크,37.393054,126.634729
728,3138.0,국제업무지구,37.399907,126.630347


In [28]:
df_bike[['대여소\n번호', "보관소(대여소)명", "위도(Y)", "경도(X)"]]

,대여소\n번호,보관소(대여소)명,위도(Y),경도(X)
0,102,망원역 1번출구 앞,37.555649,126.910629
1,103,망원역 2번출구 앞,37.554951,126.910835
2,104,합정역 1번출구 앞,37.550629,126.914986
3,105,합정역 5번출구 앞,37.550007,126.914825
4,106,합정역 7번출구 앞,37.548645,126.912826
...,...,...,...,...
2581,5070,마곡13단지힐스테이트마스터 후문(1320동 앞),37.556763,126.818695
2582,5072,김포공항입구 교통섬,37.561546,126.807083
2583,5073,우장산역 헤링턴타워 앞,37.546619,126.836235
2584,5074,마곡동 주니어마곡점앞,37.558235,126.825760
